In [1]:
from tkinter import *
from tkinter import filedialog

# A function to allow the user to select the folder contianing the data.
# Function inputs args: None. 
# Function output 1: The path of that the folder selected by the user. 
def folder_selection_dialog():
    root = Tk()
    root.title('Please select the directory containing the images')
    root.filename = filedialog.askdirectory(initialdir="/", title="Select A Folder")
    directory = root.filename
    root.destroy()

    return directory

In [2]:
import os 
from PIL import Image
import numpy as np

# Function to consider existing images, then crop them to a smaller size to make training a bit easier. This function will not be used within the RUNME files.
def crop_images():
    
    # Select the directory with the images to crop. 
    directory = folder_selection_dialog()
    
    # Get a list of the images. 
    image_list = [_ for _ in os.listdir(directory) if 'C01.tif' in _]
    
    # Make their folder. 
    if not os.path.exists(os.path.join(directory, 'smaller_images')):
        os.makedirs(os.path.join(directory, 'smaller_images'))
    
    # Iterate through each of the images and get 4 smaller images from each.
    x = 1
    for i in range(len(image_list)): 
        
        # Load in the image. 
        image_i = np.array(Image.open(os.path.join(directory, image_list[i])))
        l = 1024 # This is an arbitrary value, selected for ease of building this model and developing this skill set. 
        
        # Get the top left cropped image and save it. 
        image_TL = image_i[0:l,0:l] 
        im = Image.fromarray(image_TL)
        im.save(os.path.join(directory, 'smaller_images', f'image_{str(x)}.tif'))
        x += 1
        
        # Get the top right cropped image and save it.                      
        image_TR = image_i[0:l,l:l*2]
        im = Image.fromarray(image_TR)
        im.save(os.path.join(directory, 'smaller_images', f'image_{str(x)}.tif'))
        x += 1
         
        # Get the bottom left cropped image and save it.                      
        image_BL = image_i[l:l*2,0:l]
        im = Image.fromarray(image_BL)
        im.save(os.path.join(directory, 'smaller_images', f'image_{str(x)}.tif'))
        x += 1
         
        # Get the bottom right cropped image and save it.                      
        image_BR = image_i[l:l*2,l:l*2]
        im = Image.fromarray(image_BR)
        im.save(os.path.join(directory, 'smaller_images', f'image_{str(x)}.tif'))
        x += 1

In [3]:
from tkinter import *
from tkinter import filedialog

# A function to allow the user to select the model they wish to use or retrain. 
# Function inputs args: None. 
# Function output 1: The file path of that which was selected by the user. 
def file_selection_dialog():
    root = Tk()
    root.title('Please select the machine learning model in question')
    root.filename = filedialog.askopenfilename(initialdir="/", title="Please select the machine learning model in question", filetypes=[("All files", "*.*")])
    file_path = root.filename
    root.destroy()

    return file_path

In [ ]:
import matplotlib.pyplot as plt
import os

# Function to display and save the training loss and validation loss per epoch.
# Function input arg 1: discriminator_training_loss_real --> Array of size 1 x num_epochs. This array contains the calculated values of loss for training. 
# Function input arg 2: discriminator_training_loss_fake --> Array of size 1 x num_epochs. This array contains the calculated values of loss for training. 
# Function input arg 3: generator_training_loss --> Array of size 1 x num_epochs. This array contains the calculated values of loss for training. 
# Function input arg 4: discriminator_testing_loss_real --> Array of size 1 x num_epochs. This array contains the calculated values of loss for training. 
# Function input arg 5: discriminator_testing_loss_fake --> Array of size 1 x num_epochs. This array contains the calculated values of loss for training. 
# Function input arg 6: save_data --> True or Flase. When true, saves plot to data directory.  
# Function input arg 7: display_plot --> True or Flase. When true, displays the plot. 
# Function input arg 8: directory --> The directory containing the training dataset. 
# Function input arg 9: date_time --> The datetime string in the format of 'YMD_HMS'. 
def loss_graph(discriminator_training_loss_real, 
               discriminator_training_loss_fake,
               generator_training_loss,
               discriminator_testing_loss_real,
               discriminator_testing_loss_fake,
               save_data, 
               display_plot,
               directory, 
               date_time):
    
    # Plot the loss per epoch. 
    y = list(range(0,len(discriminator_training_loss_real)))
    plt.plot(y, discriminator_training_loss_real, label = "Discriminator (training) loss real", color='blue')
    plt.plot(y, discriminator_training_loss_fake, label = "Discriminator (training) loss fake", color='dodgerblue')
    plt.plot(y, generator_training_loss, label = "Generator loss", color='fuchsia')
    plt.plot(y, discriminator_testing_loss_real, label = "Discriminator (test) loss real", color='darkred')
    plt.plot(y, discriminator_testing_loss_fake, label = "Discriminator (test) loss fake", color='indianred')

    plt.rcParams.update({'font.size': 15})
    plt.ylabel('Loss', labelpad=10) # The labelpad argument alters the distance of the axis label from the axis itself. 
    plt.xlabel('Epoch', labelpad=10)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    # Save the plot if the user desires it.
    if save_data:
        folder_name = f'training data_{date_time}'
        if not os.path.exists(os.path.join(directory, folder_name)):
            os.makedirs(os.path.join(directory, folder_name))
        file_path = os.path.join(directory, folder_name, f'loss_{date_time}.png')
        plt.savefig(file_path, dpi=200, bbox_inches='tight')
    
    # Display the plot if the user desires it. 
    if (display_plot == False):
        plt.close()
    else:
        plt.show()   

In [ ]:
import matplotlib.pyplot as plt
import os

# Function to display and save the training accuracy and validation accuracy per epoch.
# Function input arg 1: discriminator_training_accuracy_real --> Array of size 1 x num_epochs. This array contains the calculated values of loss for training. 
# Function input arg 2: discriminator_training_accuracy_fake --> Array of size 1 x num_epochs. This array contains the calculated values of loss for training. 
# Function input arg 3: discriminator_testing_accuracy_real --> Array of size 1 x num_epochs. This array contains the calculated values of loss for training. 
# Function input arg 4: discriminator_testing_accuracy_fake --> Array of size 1 x num_epochs. This array contains the calculated values of loss for training. 
# Function input arg 5: save_data --> True or Flase. When true, saves plot to data directory.  
# Function input arg 6: display_plot --> True or Flase. When true, displays the plot. 
# Function input arg 7: directory --> The directory containing the training dataset. 
# Function input arg 8: date_time --> The datetime string in the format of 'YMD_HMS'. 
def accuracy_graph(discriminator_training_accuracy_real, 
                   discriminator_training_accuracy_fake,
                   discriminator_testing_accuracy_real,
                   discriminator_testing_accuracy_fake, 
                   save_data, 
                   display_plot,
                   directory, 
                   date_time):
    
    # Plot the accuracy per epoch. 
    y = list(range(0,len(discriminator_training_accuracy_real)))
    plt.plot(y, discriminator_training_accuracy_real, label = "Discriminator (training) accuracy real", color='blue')
    plt.plot(y, discriminator_training_accuracy_fake, label = "Discriminator (training) accuracy fake", color='dodgerblue')
    plt.plot(y, discriminator_testing_accuracy_real, label = "Discriminator (test) accuracy real", color='darkred')
    plt.plot(y, discriminator_testing_accuracy_fake, label = "Discriminator (test) accuracy fake", color='indianred')

    plt.rcParams.update({'font.size': 15})
    plt.ylabel('Accuracy', labelpad=10) # The leftpad argument alters the distance of the axis label from the axis itself. 
    plt.xlabel('Epoch', labelpad=10)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    # Save the plot if the user desires it.
    if save_data:
        folder_name = f'training data_{date_time}'
        if not os.path.exists(os.path.join(directory, folder_name)):
            os.makedirs(os.path.join(directory, folder_name))
        file_path = os.path.join(directory, folder_name, f'accuracy_{date_time}.png')
        plt.savefig(file_path, dpi=200, bbox_inches='tight')
    
    # Display the plot if the user desires it. 
    if (display_plot == False):
        plt.close()
    else:
        plt.show()   

In [ ]:
import cv2 
import numpy as np
import os
from math import floor 

# A function which will append images within a directory into a numpy array.
# Function input 1: directory [string] --> The directory containing the images.
# Function input 2: file_type [string] --> The file type of the training images e.g. '.tif'.
# Function input 3: indices [list] --> The list of image indices to indicate which images and which transformations to use. 
# Function output 1: image stack [numpy array] --> The 3D stack of appended images. 
def append_images(directory,
                  file_type,
                  indices):

    # Create an empty list. 
    image_stack = []
    
    # Iterate through the images of our list and append them to our stack. 
    for i in range(len(indices)):
        
        # Determine which unmodified image we need to use. 
        image_number = floor(indices[i]/8)+1
        image_name = f"image_{str(image_number)}{file_type}"
        
        file_path = os.path.join(directory, image_name)
        img = cv2.imread(file_path, -1)
        
        # Scale the image between -1 and 1. 
        img = np.interp(img, (img.min(), img.max()), (-1, +1))
        
        # Determine which transformation to apply. 
        transformation_1 = (indices[i]) % 8
        transformation_2 = (indices[i]) % 4
        
        # Rotation
        if 0 <= transformation_1 <= 3:
            img = np.rot90(img, 1)
        
        # Horizontal flipping.
        if 1 <= transformation_2 <= 2:
            img = np.flip(img, axis=0) 
        
        # Vertical flipping.
        if 2 <= transformation_2 <= 3:
            img = np.flip(img, axis=1) 

        # Add an extra axis (for processing later) and append our image to the stack.
        img = np.stack((img,)*1, axis=-1)
        image_stack.append(img)

    # Convert the stack to a numpy array. 
    image_stack = np.asarray(image_stack)

    return image_stack 

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Reshape, Conv2DTranspose, Conv2D, Dropout, BatchNormalization

# Funtion to create our generator. 
# Function input 1: latent_length [int] --> Size of the 1D latent space vector. 
# Function input 2: img_height [int] --> Image height in pixels. 
# Function input 3: img_width [int] --> Image width in pixels. 
# Function output 1: model --> The untrained model. 
def create_generator(latent_length,
                     img_height,
                     img_width):
    
    # Create the backbone for our sequential model.
    model = Sequential()
    
    # First, we need to create smaller 4x4 images which can be upscaled.
    # To add redundancy to our model, we'll make 'i' such images. 
    n_nodes = 4 * 4 * 32
    model.add(Dense(n_nodes, input_dim=latent_length))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    
    # We then need to reshape this Dense layer from a single dimension to 3 dimensions. 
    model.add(Reshape((4,4,32)))
    
    # Updample to 8x8: use 32 differnt convolutions, of size 4x4. 
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding='same'))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Upsample to 16x16: use 32 differnt convolutions, of size 4x4. 
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding='same'))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Upsample to 32x32: use 32 differnt convolutions, of size 4x4. 
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding='same'))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Upsample to 64x64: use 32 differnt convolutions, of size 4x4. 
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding='same'))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Upsample to 128x128: use 32 differnt convolutions, of size 4x4. 
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding='same'))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Upsample to 256x256: use 32 differnt convolutions, of size 4x4. 
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding='same'))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Upsample to 512x512: use 32 differnt convolutions, of size 4x4. 
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding='same'))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(BatchNormalization(momentum=0.8))
    
    # Upsample to 1024x1024: use 32 differnt convolutions, of size 4x4. 
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding='same'))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Create our output layer.
    # Use a tanh activation to scale our pixel values between -1 and +1. 
    model.add(Conv2D(1, (3,3), activation='tanh', padding='same'))
    
    # Return the model as the function output. 
    return model 

In [ ]:
import numpy as np 

# Function to create the latent space (the generator's input). 
# Function input 1: latent_length [int] --> The size of the latent dimension for a single sample. In my case, it's 100.
# Function input 2: number_of_samples [int] --> The number of samples for which we need to create a latent space. 
# Function output: latent_space [array] --> The latent spaces for all the samples. 
def create_latent_space(latent_length,
                        number_of_samples): 
    
    # First, we must create random numbers adhering to a normal (gaussian) distribution.
    latent_space = np.random.randn(latent_length * number_of_samples)
    
    # Reshape the vector or numbers to a 2D array, such that it can be used to represent numerous samples.
    latent_space = latent_space.reshape(number_of_samples, latent_length)
    
    return latent_space

In [ ]:
import numpy as np 

# Function to create fake images using our generator. 
# Function input 1: generator_model --> The generator model.
# Function input 2: latent_length [int] --> The size of the latent dimension for a single sample. In my case, it's 100.
# Function input 3: number_of_samples [int] --> The number of samples for which we need to create a latent space. 
def make_fake_images(generator_model,
                     latent_length,
                     number_of_samples):
    
    # Create the latent space(s).
    latent_space = create_latent_space(latent_length, number_of_samples)
    
    # Use our model to create fake images. 
    fake_images = generator_model.predict(latent_space)
    
    # Create the corresponding 'fake' class labels (0).
    fake_image_labels = np.zeros((number_of_samples, 1))
    
    return fake_images, fake_image_labels

In [ ]:
import random
import numpy as np

# Function to select and return a subset of the real images. 
# Function input 1: num_requested_images [int] --> The number of images you need from the total array. 
# Function input 2: unused images_indices [list] --> The unused image indexes.
# Function output 1: unused_images_indices [list] --> List of image indices which have not yet been used.
# Function output 2: real_images_subset [array] --> Array of the real images, organised as [number_of_images, heigth, width, channels].
# Function output 3: real_images_labels [array] --> Array of 1s, of the same length as the number of images.
def return_real_images(num_requested_images,
                       unused_images_indices):
    
    # Get the minimum and maximum.
    number_of_idx = len(unused_images_indices)
    
    # If we have sufficient images to take a selection from. 
    if num_requested_images < len(unused_images_indices):
        
        # Get the indices to select the indices.
        indices = random.sample(range(number_of_idx), num_requested_images)
        
        # Get the corresponding images. 
        real_images_subset = append_images(directory,
                                           file_type,
                                           indices)
        
        # Remove these indices from the unused_images_indices. 
        unused_images_indices = [_ for _ in unused_images_indices if _ not in indices]

        # Get the real image labels.
        real_images_labels = np.ones(real_images_subset.shape[0]) 
        
    # If we have an insifficient number of images, take whatever is left. 
    else:
    
        # Get the rest of the real images. 
        real_images_subset = append_images(directory,
                                           file_type,
                                           unused_images_indices)

        # Clear out unused_images_indices.
        unused_images_indices = [] 
        
        # Get the real image labels.
        real_images_labels = np.ones(real_images_subset.shape[0]) 
        
    return unused_images_indices, real_images_subset, real_images_labels

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, LeakyReLU, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Function to create the discriminator. 
# Function input arg 1: img_height [int] --> The pixel height of the image.
# Function input arg 2: img_width [int] --> The pixels width of the image. 
# Function input arg 3: img_channels [int] --> The number of channels to the image. 
def create_discriminator(img_height,
                         img_width,
                         img_channels): 
    
    # Define the input shape of our images. 
    image_shape = (img_height, img_width, img_channels)
    
    # Create an instance of a sequential model. 
    model = Sequential()
    
    # Create a normal convolutional layer. 
    model.add(Conv2D(32, (3,3), padding='same', input_shape=image_shape))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    
    # Downsample our input to 512 x 512. 
    model.add(Conv2D(32, (3,3), strides=(2,2), padding='same', input_shape=image_shape))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))

    # Downsample our input to 256 x 256. 
    model.add(Conv2D(32, (3,3), strides=(2,2), padding='same', input_shape=image_shape))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    
    # Downsample our input to 128 x 128. 
    model.add(Conv2D(32, (3,3), strides=(2,2), padding='same', input_shape=image_shape))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))

    # Downsample our input to 64 x 64. 
    model.add(Conv2D(32, (3,3), strides=(2,2), padding='same', input_shape=image_shape))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
        
    # Downsample our input to 32 x 32. 
    model.add(Conv2D(64, (3,3), strides=(2,2), padding='same', input_shape=image_shape))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    
    # Downsample our input to 16 x 16. 
    model.add(Conv2D(64, (3,3), strides=(2,2), padding='same', input_shape=image_shape))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))

    # Downsample our input to 8 x 8. 
    model.add(Conv2D(64, (3,3), strides=(2,2), padding='same', input_shape=image_shape))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    
    # Downsample our input to 4 x 4. 
    model.add(Conv2D(64, (3,3), strides=(2,2), padding='same', input_shape=image_shape))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha=0.2))
    
    # Create the classifier. 
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model. 
    optim = Adam(learning_rate=0.002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    
    return model

In [ ]:
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam

# Function to combine the generator and the discriminator, such that the generator can be trained. 
# Function input 1: [model instance] --> the 
def create_GAN(generator_model, 
               discriminator_model): 
    
    # First, prevent the discriminator from training.
    discriminator_model.trainable = False 
    
    # Combine the two models with a sequential model. 
    model = Sequential()
    model.add(generator_model)
    model.add(discriminator_model)
    
    # Compile the model.
    optim = Adam(learning_rate=0.002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=optim)
    
    return model

In [ ]:
from math import ceil 
import pandas as pd
import os as os 
from tqdm import trange

# Function to train the generator and the discriminator.
# Function input arg 1: generator_model [model] --> The generator. 
# Function input arg 2: discriminator_model [model] --> The discriminator. 
# Function input arg 2: gan_model [model] --> The GAN. 
# Function input arg 3: latent_length [int] --> The size of the latent space. 
# Function input arg 4: num_epochs [int] --> The number of epochs to train for. 
# Function input arg 5: batch_size [int, preferbly even] --> Num images processed per batch. NB: Your batch size can never equal more than 2*
# Function input arg 6: directory [string] --> The directory containing the training images. 
# Fucntion input arg 7: file_type [string] --> The file type of the training images. 
# Function input arg 8: test_train_split [float] --> The fraction of the data which will be used for training. e.g. 0.8 would mean 80% be used for training, while 20% for testing.
# Function input arg 9: date_time [string] --> A date time string organised as 'YYYYMMDD_HHMMSS'. 
# Function input arg 10: training_data [DataFrame] --> The pandas DataFrame of training data.
# Function input arg 10: save_data [bool] --> When True, saves the models and the training data.
# Function output 1: training_Data [DataFrame] --> The pandas DataFrame of training data. 
def train_GAN(generator_model, 
              discriminator_model,
              gan_model,
              latent_length,
              num_epochs,
              batch_size, 
              directory, 
              file_type, 
              test_train_split,
              date_time, 
              save_data, 
              training_data):
    
    # Re-create the list of indexes for the images (which haven't been used for training) at the start of each epoch (it changes with each batch). 
    unused_images_indices = []
    img_in_dir = len([_ for _ in os.listdir(directory) if file_type in _])
    for i in range(img_in_dir*8):
        unused_images_indices.append(i)
    random.shuffle(unused_images_indices)
    
    # Get the list of image (indices) for testing. 
    test_unused_images_indices = unused_images_indices[(int(test_train_split * len(unused_images_indices))):len(unused_images_indices)]

    # Calculate the number of batches per epoch.
    num_batches = int((test_train_split * len(unused_images_indices)) / (batch_size * 0.5))
    
    # Loop through each epoch and train the model. 
    for i in (x := trange(num_epochs)):
        print(f'Epoch: {str(i+1)} of {str(num_epochs)}')
        # Set the description for the trange progress bar. 
        x.set_description(f"Training GAN model. Epoch number:{str(i)}")
          
        # Create / reset the image indices.
        train_unused_images_indices = unused_images_indices[0:(int(test_train_split * len(unused_images_indices)))]

        # Create empty lists for storing training data. 
        batch_discriminator_train_loss_real = []
        batch_discriminator_train_loss_fake = []
        batch_discriminator_train_accuracy_real = []
        batch_discriminator_train_accuracy_fake = [] 
        batch_generator_train_loss = []
        
        batch_discriminator_test_loss_real = []
        batch_discriminator_test_loss_fake = []
        batch_discriminator_test_accuracy_real = []
        batch_discriminator_test_accuracy_fake = [] 
        
        # Loop though each batch and train the model.
        for t in range(num_batches): 

            print(f'   Batch: {str(t+1)} of {str(num_batches)}')
            ###########################
            # TRAIN THE DISCRIMINATOR.
            ###########################
            
            # Select a half-batch of real data and update the discriminator. 
            number_of_samples = 1
            
            train_unused_images_indices, real_img_subset, real_img_labels = return_real_images(number_of_samples,
                                                                                               train_unused_images_indices)
            
            d_loss_real, d_accuracy_real = discriminator_model.train_on_batch(real_img_subset, 
                                                                              real_img_labels)            
            print(1)
            # Select a half-batch of fake data and update the discriminator. 
            if real_img_subset.shape[0] < number_of_samples:
                number_of_samples = real_images_subset.shape[0] # This checks to see whether we have enough real images to 'fill' the bacth. If not, then we create a smaller number of fake images, equal to the number of real images. We don't want to imbalance the model
            
            del real_img_subset, real_img_labels # Save memory.

            fake_images, fake_image_labels = make_fake_images(generator_model,
                                                              latent_length,
                                                              number_of_samples)   
                      
            d_loss_fake, d_accuracy_fake = discriminator_model.train_on_batch(fake_images, 
                                                                              fake_image_labels)
            print(2)
            del fake_images, fake_image_labels # Save memory

            # Add the discriminator accuracy and loss to our lists.
            batch_discriminator_train_loss_real.append(d_loss_real)
            batch_discriminator_train_loss_fake.append(d_loss_fake)
            batch_discriminator_train_accuracy_real.append(d_accuracy_real)
            batch_discriminator_train_accuracy_fake.append(d_accuracy_fake)
            
            ######################
            # TRAIN THE GENERATOR.
            ######################
            
            # Create the latent space. 
            latent_space = create_latent_space(latent_length,
                                               1)
            
            # Create array of labels... BUT... label as '1', despite the fact that they are fake. 
            fake_image_labels = np.ones(1)
            
            with tf.Session() as sess:

                # Feed it into the GAN.
                generator_loss = gan_model.train_on_batch(latent_space, 
                                                          fake_image_labels)
            print(3)
            # Add the generator loss to our list.
            batch_generator_train_loss.append(generator_loss)
            
            #############################
            # EVALUATE THE DISCRIMINATOR.
            #############################

            number_of_samples = 1

            # Get the real images. 
            _, test_images, test_images_labels = return_real_images(number_of_samples,
                                                                    test_unused_images_indices)

            # Evaluate using the real images. 
            d_loss_real, d_accuracy_real = discriminator_model.evaluate(test_images, 
                                                                        test_images_labels,
                                                                        verbose = 0)
            print(4)
            del test_images, test_images_labels # Save memory

            # Get the fake images.
            test_images, test_images_labels = make_fake_images(generator_model,
                                                               latent_length,
                                                               number_of_samples)   

            d_loss_fake, d_accuracy_fake = discriminator_model.evaluate(test_images, 
                                                                        test_images_labels,
                                                                        verbose = 0)

            del test_images, test_images_labels # Save memory
            
            batch_discriminator_test_loss_real.append(d_loss_real)
            batch_discriminator_test_loss_fake.append(d_loss_fake)
            batch_discriminator_test_accuracy_real.append(d_accuracy_real)
            batch_discriminator_test_accuracy_fake.append(d_accuracy_fake)
            
            print(5)
            
            # If we're at the end of the last batch iteration.
            if t == (num_batches-1):
                
                ###############
                # LOG THE DATA.
                ###############
                
                # Make an empty list.
                epoch_data = []
                
                # Get the mean values of loss and accuracy. 
                length = len(batch_discriminator_train_loss_real)
                epoch_data.append(sum(batch_discriminator_train_loss_real) / length)
                epoch_data.append(sum(batch_discriminator_train_loss_fake) / length)
                epoch_data.append(sum(batch_discriminator_train_accuracy_real) / length)
                epoch_data.append(sum(batch_discriminator_train_accuracy_fake) / length)
                epoch_data.append(sum(batch_generator_train_loss) / length)
                
                epoch_data.append(sum(batch_discriminator_test_loss_real) / length)
                epoch_data.append(sum(batch_discriminator_test_loss_fake) / length)
                epoch_data.append(sum(batch_discriminator_test_accuracy_real) / length)
                epoch_data.append(sum(batch_discriminator_test_accuracy_fake) / length)

                # Add the list to the pandas dataframe. 
                training_data.loc[len(training_data)] = epoch_data
    
    # Now that the training is done, save the training data and models, if the user desires it. 
    if save_data == True: 
        
        # Create the directory if it doesn't already exist. 
        folder_name = f'training data_{date_time}'
        if not os.path.exists(os.path.join(directory, folder_name)):
            os.makedirs(os.path.join(directory, folder_name))
        
        # Save the training data. 
        file_path = os.path.join(directory, folder_name, 'training_log.csv')
        training_data.to_csv(file_path, index=False)
        
        # Save the models. 
        file_path = os.path.join(directory, folder_name, f"GAN_{date_time}.hdf5")
        gan_model.save(file_path) 
        file_path = os.path.join(directory, folder_name, f"discriminator_{date_time}.hdf5")
        discriminator_model.save(file_path) 
        file_path = os.path.join(directory, folder_name, f"generator_{date_time}.hdf5")
        generator_model.save(file_path) 
        
    # Return function outputs. 
    return training_data

In [40]:
from datetime import datetime
import os 
from keras.utils.vis_utils import plot_model
import pydot 
import pydotplus
import graphviz
import time 
import tensorflow as tf 
from keras.models import load_model 
import pandas 
from pandas import read_csv

# Function to train the GAN and save the model outputs. 
# Function input arg 1: directory [string] --> The directory containing the training data.
# Function input arg 2: file_type [string] --> The file type of the training data e.g. '.tif'.
# Function input arg 3: save_data [bool] --> When True, saves the models and the training data.
# Function input arg 4: num_epochs [int] --> The number of epochs to train for. 
# Function input arg 5: batch_size [int, preferbly even] --> Num images processed per batch. NB: Your batch size can never equal more than 2*
# Function input arg 6: test_train_split [float] --> The fraction of the data which will be used for training. e.g. 0.8 would mean 80% be used for training, while 20% for testing.
# Function input arg 7: display_plot [bool] --> When True, prints the graphs of accuracy and loss.
# Function input arg 8: train_previous_model [bool] --> When true, the code will continue training a saved model of the user's choice. 
def train_model(directory,
                file_type = '.tif',
                save_data = True,
                num_epochs = 2,
                batch_size = 2,
                test_train_split = 0.003,
                display_plot=True, 
                train_previous_model=False):
    
    ### (1) Establish variables useful for the rest of the code. 
    
    now = datetime.now()
    date_time = now.strftime("%Y%m%d_%H%M%S")
    
    # I set up my GPU to run the model, but it doesn't have enough memory, so I'm switching back to CPU. 
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

    ### (2) Load in a saved model to continue it's training, or make a new model. 
    
    # Create and combine our models to make the GAN.
    if train_previous_model == False: 
        
        # Create the discriminator. 
        img = cv2.imread(os.path.join(directory, 'image_1.tif'), -1)
        img_height = img.shape[0]
        img_width = img.shape[1]
        img_channels = 1
        discriminator_model = create_discriminator(img_height, 
                                                   img_width, 
                                                   img_channels)

        # Create the generator. 
        latent_length = 100 
        generator_model = create_generator(latent_length,
                                           img_height, 
                                           img_width)

        # Create the GAN.
        gan_model = create_GAN(generator_model, discriminator_model)
        
        # Create an empty pandas dataframe to store the training information.
        training_data = pd.DataFrame(columns=['Discriminator training loss real', 'Discriminator training loss fake', 'Discriminator training accuracy real', 'Discriminator training accuracy fake', 'Generator training loss', 'Discriminator testing loss real', 'Discriminator testing loss fake', 'Discriminator testing accuracy real', 'Discriminator testing accuracy fake'])
     
    # Load in the saved model to continue it's training. 
    elif train_previous_model == True: 
        
        # Ask the user to select the folder contining the (semi)trained models. 
        trained_directory = folder_selection_dialog()
        
        # Load the generator.
        generator_name = [_ for _ in os.listdir(trained_directory) if 'generator' in _]
        generator_model = load_model(os.path.join(trained_directory, generator_name[0]))
        
        # Load the discriminator.
        discriminator_name = [_ for _ in os.listdir(trained_directory) if 'discriminator' in _]
        discriminator_model = load_model(os.path.join(trained_directory, discriminator_name[0]))
    
        # Create the GAN.
        gan_model = create_GAN(generator_model, discriminator_model)

        # Load the training data. 
        training_data = read_csv(os.path.join(trained_directory, 'training_log.csv'))
        
    ### (3) Plot and save the model architechtures should the user desire it.
    folder_name = f'training data_{date_time}'
    if save_data == True: 

        # If the folder to contain training graph etc. doesn't exist, create it. 
        if not os.path.exists(os.path.join(directory, folder_name)):
            os.makedirs(os.path.join(directory, folder_name))

        # Save the model architecture. 
        os.chdir(os.path.join(directory, folder_name))

        file_name = f'discriminator_flow_chart_{date_time}.png'
        plot_model(discriminator_model, to_file=file_name, show_shapes=True, show_layer_names=True)

        file_name = f'generator_flow_chart_{date_time}.png'
        plot_model(generator_model, to_file=file_name, show_shapes=True, show_layer_names=True)

        file_name = f'GAN_flow_chart_{date_time}.png'
        plot_model(gan_model, to_file=file_name, show_shapes=True, show_layer_names=True)

    ### (3) Train the GAN. 
    start = time.time()

    training_data = train_GAN(generator_model, 
                              discriminator_model,
                              gan_model,
                              latent_length,
                              num_epochs,
                              batch_size, 
                              directory, 
                              file_type, 
                              test_train_split,
                              date_time, 
                              save_data,
                              training_data)
    
    end = time.time()
    print(f'{((end-start)/60)}_min')
    print(f'{((end-start)/60)/60}_hours')
    
    ### (4) Use the training_data to plot graphs of the model's loss and accuracy per epoch.
        
    # Plot the loss. 
    loss_graph(training_data['Discriminator training loss real'], 
               training_data['Discriminator training loss fake'],
               training_data['Generator training loss'],
               training_data['Discriminator testing loss real'],
               training_data['Discriminator testing loss fake'],
               save_data, 
               display_plot,
               directory, 
               date_time)
    
    # Plot the accuracy. 
    accuracy_graph(training_data['Discriminator training accuracy real'], 
                   training_data['Discriminator training accuracy fake'],
                   training_data['Discriminator testing accuracy real'],
                   training_data['Discriminator testing accuracy fake'],
                   save_data, 
                   display_plot,
                   directory, 
                   date_time)
    
    ### (5) Print the completion statement. 
    class bcolors:
        OKBLUE = '\033[94m'   
        ENDC = '\033[0m'

    print(f'▇▇▇▇▇▇▇▇▇▇▇▇\nThank you for using this code. The training proces is now complete.\n\nPlease refer to the following directory to find your training data:\n\n{bcolors.OKBLUE}{os.path.join(directory, folder_name)}{bcolors.ENDC}\n▇▇▇▇▇▇▇▇▇▇▇▇')

In [ ]:
from tqdm import trange 
import numpy as np
import os 
import keras 
from keras.models import load_model 
from datetime import datetime 
from PIL import Image 

# Function to use a trained generator to create our fake data. 
# Function input arg 1: number_of_images [int] --> The number of images you wish to generate. 
def use_generator(number_of_images):
    
    #### (1) Establish variables important for the code. 
    now = datetime.now()
    date_time = now.strftime("%Y%m%d_%H%M%S")
    
    #### (2) Load in the previously trained model.
    previous_model_path = file_selection_dialog()
    generator_model = load_model(previous_model_path, compile=False)
    
    # Get the parent directory.
    directory, _ = os.path.split(previous_model_path)
            
    # Make the new directory to store the fake images.
    if not os.path.exists(os.path.join(directory, 'fake_images')):
        os.makedirs(os.path.join(directory, 'fake_images'))
        
    #### (3) Make and save the images.   
    latent_length = 100
    for i in (x := trange(number_of_images)):
        
        # Set the description for the trange progress bar. 
        x.set_description(f"Saving fake images:{str(i)}")
        
        # Make the fake image.
        latent_space = create_latent_space(latent_length, 1)
        fake_image = generator_model(latent_space)
        fake_image = fake_image[0,:,:,0]
        fake_image = (255*(fake_image - np.amin(fake_image))/np.ptp(fake_image)).astype(int)        

        # Save the fake image.
        file_name = f'fake_image{str(i)}.png'
        file_path = os.path.join(directory, 'fake_images', file_name)
        im = Image.fromarray(fake_image)
        im.save(file_path)
    
    #### (4) Print a completion statement. 
    class bcolors:
        OKBLUE = '\033[94m'   
        ENDC = '\033[0m'
    print(f'▇▇▇▇▇▇▇▇▇▇▇▇\nThank you for using this code. The fake images have been made by the generator.\n\nPlease refer to the following directory to find them:\n\n{bcolors.OKBLUE}{os.path.join(directory)}{bcolors.ENDC}\n▇▇▇▇▇▇▇▇▇▇▇▇')

In [ ]:
# Figure out the min number of images neede within dataset for code to work.

# Consider removing .predict to avoid the retracing warnings. 